# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [2]:
# your code here
import pandas as pd
import random

In [3]:
col_names = ['target', 'id', 'date', 'flag', 'user', 'text']

In [4]:
df = pd.read_csv("../data/training.1600000.processed.noemoticon.csv", encoding='latin1', header=None, names=col_names)

In [5]:
df_sample = df.sample(2000)

In [6]:
#clean_up
import re
def clean_up(s):
    s_clean = re.sub(r"https?:/[^\s]*", '', s)
    s_clean = re.sub(r"[\W\d]", ' ', s_clean)
    return s_clean.lower()

In [7]:
#tokenize
from nltk import word_tokenize
def tokenize(s):
    return word_tokenize(s)

In [8]:
#stem_and_lemmatize
from nltk.stem import WordNetLemmatizer
def stem_and_lemmatize(l):
    stemmer = SnowballStemmer("english")
    stemmed_list = [stemmer.stem(i) for i in l]
    lemmatizer = WordNetLemmatizer()
    stemmed_and_lemmatized_list = [lemmatizer.lemmatize(i) for i in stemmed_list]
    return stemmed_and_lemmatized_list

In [9]:
#stopwords
from nltk.corpus import stopwords
def remove_stopwords(l):
    stop_words = set(stopwords.words('english'))
    return [w for w in l if w not in stop_words]

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [10]:
# your code here
df_sample['text_processed'] = df_sample['text'].apply(clean_up)
df_sample['text_processed'] = df_sample['text_processed'].apply(tokenize, stem_and_lemmatize)
df_sample['text_processed'] = df_sample['text_processed'].apply(remove_stopwords)
df_sample

,target,id,date,flag,user,text,text_processed
1492016,4,2069211898,Sun Jun 07 15:28:01 PDT 2009,NO_QUERY,jblagg,"taylor swift/keith urban boston, MA -- 80% sur...","[taylor, swift, keith, urban, boston, sure, im..."
394024,0,2055537258,Sat Jun 06 09:30:00 PDT 2009,NO_QUERY,nikkluv,@Sammeroni hi bebehhhh &lt;3 i miss talking to...,"[sammeroni, hi, bebehhhh, lt, miss, talking, h..."
1025936,4,1883620093,Fri May 22 08:50:32 PDT 2009,NO_QUERY,LdyDisney,@SDRadio Hee! Thanks Chris for the #followfri...,"[sdradio, hee, thanks, chris, followfriday, me..."
239300,0,1980593098,Sun May 31 07:10:04 PDT 2009,NO_QUERY,anonesque,@failwatcher how do you not fail at assassinat...,"[failwatcher, fail, assassinating, people, rin..."
1195806,4,1984684191,Sun May 31 15:35:51 PDT 2009,NO_QUERY,Doodlebug18,@Heidi_range Just wanted to say that you and t...,"[heidi_range, wanted, say, girls, amazing, sum..."
...,...,...,...,...,...,...,...
165943,0,1961099229,Fri May 29 08:59:07 PDT 2009,NO_QUERY,tinkerdinkers,im tweeting... this is so hard... i dont get i...,"[im, tweeting, hard, dont, get]"
184602,0,1967748376,Fri May 29 20:08:31 PDT 2009,NO_QUERY,bnwhite89,is having fun all wrong,"[fun, wrong]"
71838,0,1694068858,Mon May 04 00:56:51 PDT 2009,NO_QUERY,chelseydee,Just realized that when I started freaking out...,"[realized, started, freaking, thought, sheets,..."
1117520,4,1973435545,Sat May 30 11:08:33 PDT 2009,NO_QUERY,bengillam,@daveizzle guess so. My worry was that the fil...,"[daveizzle, guess, worry, files, would, stay, ..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [11]:
# your code here
from nltk.probability import FreqDist

In [12]:
all_text_processed = []
for l in df_sample["text_processed"]:
    for w in l:
        all_text_processed.append(w)
    fdist = FreqDist
    fdist = FreqDist(w for w in all_text_processed)

In [13]:
#top 5K words
top_5k = [i[0] for i in fdist.most_common(5000)]
print(top_5k)

['good', 'day', 'like', 'get', 'quot', 'go', 'love', 'work', 'time', 'going', 'got', 'lol', 'today', 'really', 'u', 'one', 'know', 'night', 'thanks', 'back', 'see', 'home', 'im', 'na', 'amp', 'feel', 'want', 'miss', 'would', 'oh', 'still', 'hope', 'sleep', 'need', 'great', 'much', 'twitter', 'sorry', 'new', 'think', 'last', 'morning', 'well', 'weekend', 'sad', 'come', 'way', 'getting', 'tomorrow', 'wish', 'bad', 'haha', 'awesome', 'gon', 'though', 'tonight', 'bed', 'could', 'right', 'fun', 'first', 'already', 'watching', 'happy', 'x', 'hey', 'ever', 'guys', 'tired', 'best', 'hate', 'working', 'nice', 'phone', 'ready', 'missed', 'hours', 'week', 'even', 'ok', 'ã', 'little', 'watch', 'days', 'wan', 'say', 'cool', 'school', 'better', 'cant', 'soon', 'show', 'ì', 'long', 'yeah', 'pretty', 'next', 'bit', 'done', 'made', 'ñ', 'ð', 'people', 'us', 'never', 'another', 'make', 'wait', 'thought', 'life', 'wow', 'food', 'man', 'lt', 'friday', 'feeling', 'left', 'everyone', 'someone', 'things', 'b

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [14]:
# your code here
word_features = [x[0] for x in top_5k]
for i in word_features:
    df_sample[i]=[i in x for x in df_sample['text_processed']]

In [15]:
df_sample.drop(['flag','id','date','text'], axis=1, inplace=True)

In [16]:
#target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

In [17]:
df_sample["positive"] = df_sample["target"].apply(lambda t: False if t == 0 else True)
df_sample["positive"].value_counts()

False    1017
True      983
Name: positive, dtype: int64

In [18]:
#dict
cols = top_5k
words = []
for index,row in df_sample.iterrows():
    words_dic = {}
    for i in cols:
        words_dic[i] = i in row["text_processed"]
        words.append((words_dic, row["positive"]))

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [19]:
# your code here
import nltk

In [30]:
print(len(words))

10000000


In [31]:
#train, test
train = words[:800]
test = words[800:]

In [32]:
#train model
classifier = nltk.NaiveBayesClassifier.train(train)

In [33]:
# Most informative features
classifier.show_most_informative_features()

Most Informative Features


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [36]:
# your code here
#nltk.classify.accuracy(classifier, test)

In [38]:
"""I'm having problems trying the get the accuracy number due to my jupyter's memory space"""

"I'm having problems trying the get the accuracy number due to my jupyter's memory space"

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here